In [1]:
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# import the data set 
ds, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_ds, test_ds = ds['train'], ds['test']

In [5]:
metadata

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='C:\\Users\\AEsir\\tensorflow_datasets\\fashion_mnist\\3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-077

In [13]:
labels=metadata.features['label']
labels.names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [14]:
train_size = metadata.splits['train'].num_examples
test_size = metadata.splits['test'].num_examples

In [46]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_ds = train_ds.map(normalize).cache()
test_ds = test_ds.map(normalize).cache()

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu,
                          input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation=tf.nn.relu,
                          input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [20]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [47]:
BATCH_SIZE = 32
train_ds_batch = train_ds.cache().repeat().shuffle(train_size).batch(BATCH_SIZE)
test_ds_batch = test_ds.cache().cache().batch(BATCH_SIZE)

In [24]:
import math

In [48]:
model.fit(train_ds_batch, epochs=5, steps_per_epoch=math.ceil(train_size/BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 44s 23ms/step - loss: 0.4279 - accuracy: 0.8397
Epoch 2/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.2841 - accuracy: 0.8934
Epoch 3/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.2417 - accuracy: 0.9108
Epoch 4/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.2090 - accuracy: 0.9231
Epoch 5/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.1826 - accuracy: 0.9331


In [49]:
test_loss, test_accuracy = model.evaluate(test_ds_batch, steps = math.ceil(test_size/BATCH_SIZE))
print('test accuracy is {}'.format(test_accuracy))

313/313 [==============================] - 2s 6ms/step - loss: 0.2509 - accuracy: 0.9130
test accuracy is 0.9129999876022339


In [50]:
# for training epoch as 1
model.fit(train_ds_batch, epochs=1, steps_per_epoch=math.ceil(train_size/BATCH_SIZE))

1875/1875 [==============================] - 41s 22ms/step - loss: 0.1581 - accuracy: 0.9416


In [51]:
test_losses, test_accuracy = model.evaluate(test_ds_batch, steps = math.ceil(test_size/BATCH_SIZE))

313/313 [==============================] - 2s 6ms/step - loss: 0.2623 - accuracy: 0.9086


In [33]:
def diff_neurons(neuron_num):
    print('number of neurons in the dense layer: {}'.format(neuron_num))
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(neuron_num, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
     metrics=['accuracy'])
    model.fit(train_ds_batch, epochs=5, steps_per_epoch=math.ceil(train_size / BATCH_SIZE))
    test_losses, test_accuracy = model.evaluate(test_ds_batch, steps = math.ceil(test_size / BATCH_SIZE))
    return test_accuracy

In [34]:
neuron_array = [10, 100, 256, 512]
accuracy = []
for num in neuron_array:
    accuracy.append(diff_neurons(num))

number of neurons in the dense layer: 10
Epoch 1/5
1875/1875 [==============================] - 29s 15ms/step - loss: 0.6174 - accuracy: 0.7821
Epoch 2/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.3508 - accuracy: 0.8759
Epoch 3/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.3017 - accuracy: 0.8931
Epoch 4/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.2685 - accuracy: 0.9049
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.2957 - accuracy: 0.8967
number of neurons in the dense layer: 100
Epoch 1/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.4156 - accuracy: 0.8504
Epoch 2/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.2627 - accuracy: 0.9046
Epoch 3/5
1875/1875 [==============================] - 35s 19ms/step - loss: 0.2174 - accuracy: 0.9201
Epoch 4/5
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1865 - accuracy: 0

In [36]:
import pandas as pd

In [37]:
data = {'num of neurons in first dense layer': neuron_array, 'accuracy': accuracy}
pd.DataFrame(data=data)

,num of neurons in first dense layer,accuracy
0,10,0.8967
1,100,0.9169
2,256,0.9183
3,512,0.9176


In [38]:
def diff_neurons_add_layer(neuron_num):
    print('number of neurons in the dense layer: {}'.format(neuron_num))
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(neuron_num, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
     metrics=['accuracy'])
    model.fit(train_ds_batch, epochs=5, steps_per_epoch=math.ceil(train_size / BATCH_SIZE))
    test_losses, test_accuracy = model.evaluate(test_ds_batch, steps = math.ceil(test_size / BATCH_SIZE))
    return test_accuracy

In [39]:
neuron_array = [10, 50, 128, 180]
accuracy = []
for num in neuron_array:
    accuracy.append(diff_neurons_add_layer(num))

number of neurons in the dense layer: 10
Epoch 1/5
1875/1875 [==============================] - 39s 20ms/step - loss: 0.4700 - accuracy: 0.8322
Epoch 2/5
1875/1875 [==============================] - 37s 20ms/step - loss: 0.2863 - accuracy: 0.8951
Epoch 3/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.2383 - accuracy: 0.9133
Epoch 4/5
1875/1875 [==============================] - 42s 23ms/step - loss: 0.2070 - accuracy: 0.9242
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.2514 - accuracy: 0.9100
number of neurons in the dense layer: 50
Epoch 1/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.4129 - accuracy: 0.8509
Epoch 2/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.2577 - accuracy: 0.9043
Epoch 3/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.2145 - accuracy: 0.9208
Epoch 4/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1822 - accuracy: 0.

In [40]:
data = {'num of neurons in first dense layer': neuron_array, 'accuracy': accuracy}
pd.DataFrame(data=data)

,num of neurons in first dense layer,accuracy
0,10,0.9100
1,50,0.9149
2,128,0.9196
3,180,0.9148


In [41]:
# don't normalize the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu,
                          input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation=tf.nn.relu,
                          input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])
train_ds, test_ds = ds['train'], ds['test']
BATCH_SIZE = 32
train_ds_batch = train_ds.cache().repeat().shuffle(train_size).batch(BATCH_SIZE)
test_ds_batch = test_ds.cache().cache().batch(BATCH_SIZE)
model.fit(train_ds_batch, epochs=5, steps_per_epoch=math.ceil(train_size / BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 39s 20ms/step - loss: 0.5207 - accuracy: 0.8576
Epoch 2/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.2712 - accuracy: 0.8998
Epoch 3/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.2318 - accuracy: 0.9142
Epoch 4/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.2078 - accuracy: 0.9225
Epoch 5/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1888 - accuracy: 0.9302


In [43]:
test_losses, test_accuracy = model.evaluate(test_ds_batch, steps=math.ceil(test_size/BATCH_SIZE))

313/313 [==============================] - 2s 5ms/step - loss: 0.2937 - accuracy: 0.9005


In [52]:
data = {'scenario': ['CNN(epochs=5)', 'epochs=1', "don't normalize"],
       'accuracy': ['0.9130', '0.9086', '0.9005']}
pd.DataFrame(data=data)

,scenario,accuracy
0,CNN(epochs=5),0.9130
1,epochs=1,0.9086
2,don't normalize,0.9005
